In [ ]:
install.packages('dplyr')
install.packages('tidyverse')
install.packages('caret')
install.packages('BGLR')
install.packages('stringi')
install.packages('Matrix')
install.packages('plyr')

In [2]:
require('plyr')
require("dplyr")
require('BGLR')
require('tidyverse')
require('caret')

Loading required package: plyr

Loading required package: dplyr


Attaching package: ‘dplyr’


The following objects are masked from ‘package:plyr’:

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: BGLR

Loading required package: tidyverse

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──

✔ ggplot2 3.3.0     ✔ readr   1.3.1
✔ tibble  3.0.1     ✔ purrr   0.3.4
✔ tidyr   1.0.2     ✔ stringr 1.4.0
✔ ggplot2 3.3.0     ✔ forcats 0.4.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::arrange()   masks plyr::arrange()
✖ purrr::compact()   masks plyr::compact()
✖ dplyr::count()     masks plyr::count()
✖ dplyr::failwith()  masks plyr::failwith()
✖ dplyr::filter()    masks stats::filter()
✖ dpl

In [2]:
setwd('gxe-gee-lmm')

ERROR: Error in setwd("gxe-gee-lmm"): cannot change working directory


In [3]:
getwd()

[1] "/home/jupyter/gxe-gee-lmm"

In [5]:
install.packages('benchmarkme')

Installing package into ‘/home/jupyter/.R/library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘benchmarkmeData’, ‘doParallel’




In [6]:
library(benchmarkme)
get_ram()

1.02 TB

In [ ]:
## Import Data and Prep for Analysis

In [3]:
# Import CSV files of target data
y_train <- data.frame(read.csv('y_train_scaled.csv'))
y_test <- data.frame(read.csv('y_test_scaled.csv'))

In [4]:
y_train <- arrange(y_train, Unique_ID)
y_test <- arrange(y_test, Unique_ID)

In [ ]:
head(y_train)

In [5]:
rownames(y_train) = y_train$Unique_ID
rownames(y_test) = y_test$Unique_ID
y_train = subset(y_train, select = -c(Unique_ID))
y_test = subset(y_test, select = -c(Unique_ID))

In [ ]:
head(y_train)

In [ ]:
head(y_test)

In [ ]:
print(dim(y_train))
print(dim(y_test))

In [6]:
# Import CSV files of environmental data for CIMMYT and GEE
W_CIMMYT_train <- read.csv('X_train_scaled.csv')
W_CIMMYT_test <- read.csv('X_test_scaled.csv')
#W_gee_train <- read.csv('X_train_gee_scaled.csv')
#W_gee_test <- read.csv('X_test_gee_scaled.csv')

print(dim(W_CIMMYT_train))
print(dim(W_CIMMYT_test))
# print(dim(W_gee_train))
# print(dim(W_gee_test))

[1] 122484     59
[1] 21615    59


In [7]:
W_CIMMYT_train <- arrange(W_CIMMYT_train, Unique_ID)
W_CIMMYT_test <- arrange(W_CIMMYT_test, Unique_ID)
# W_gee_train <- arrange(W_gee_train, Unique_ID)
# W_gee_test <- arrange(W_gee_test, Unique_ID)

rownames(W_CIMMYT_train) = W_CIMMYT_train$Unique_ID
rownames(W_CIMMYT_test) = W_CIMMYT_test$Unique_ID
# rownames(W_gee_train) = W_gee_train$Unique_ID
# rownames(W_gee_test) = W_gee_test$Unique_ID

W_CIMMYT_train <- subset(W_CIMMYT_train, select = -c(Unique_ID))
W_CIMMYT_test <- subset(W_CIMMYT_test, select = -c(Unique_ID))
# W_gee_train <- subset(W_gee_train, select = -c(Unique_ID))
# W_gee_test <- subset(W_gee_test, select = -c(Unique_ID))

In [ ]:
head(W_CIMMYT_train)

In [ ]:
head(W_CIMMYT_test)

In [ ]:
print(dim(W_CIMMYT_train))
print(dim(W_CIMMYT_test))

In [8]:
# Import pedigree matrix
A <- read.csv('A_matrix_unique.csv', row.names = 1)

In [ ]:
dim(A)

In [9]:
# A matrix doesn't read in well as a CSV, need to repeat some processing steps
# Remove the X prefix from the column names
names(A) <- substring(names(A),2)
A <- A[order(rownames(A)),order(names(A))]
print(colnames(A)[1])
print(rownames(A)[1])

[1] "100507"
[1] "100507"


In [ ]:
A[1:5,1:5]

In [ ]:
dim(A)

In [10]:
X_train_GID <- read.csv('X_train_GID.csv')
X_test_GID <- read.csv('X_test_GID.csv')

In [11]:
X_train_GID <- arrange(X_train_GID, Unique_ID)
X_test_GID <- arrange(X_test_GID, Unique_ID)

rownames(X_train_GID) = X_train_GID$Unique_ID
rownames(X_test_GID) = X_test_GID$Unique_ID

X_train_GID <- subset(X_train_GID, select = -c(Unique_ID))
X_test_GID <- subset(X_test_GID, select = -c(Unique_ID))

In [ ]:
head(X_train_GID)

In [ ]:
dim(X_train_GID)

## Run Model on Sample of Data

Full dataset is too large, creates a 156Gb matrix and crashes the instance

In [ ]:
# Set Sample Indices
# train_random_sample <- sample(nrow(y_train), size = (nrow(y_train) * 1))
# test_random_sample <- sample(nrow(y_test), size = (nrow(y_test) * 1))

In [ ]:
# Sample Target Data
# y_train_sample <- as.data.frame(y_train[train_random_sample,], row.names = row.names(y_train)[train_random_sample])
# colnames(y_train_sample) <- c('GRAIN_YIELD')
# y_test_sample <- as.data.frame(y_test[test_random_sample,], row.names = row.names(y_test)[test_random_sample])
# colnames(y_test_sample) <- c('GRAIN_YIELD')

# # Sample CIMMYT Environmental Data
# W_CIMMYT_train_sample <- W_CIMMYT_train[train_random_sample,]
# W_CIMMYT_test_sample <- W_CIMMYT_test[test_random_sample,]

# # Sample GEE Environmental Data
# W_gee_train_sample <- W_gee_train[train_random_sample,]
# W_gee_test_sample <- W_gee_test[test_random_sample,]

# # Sample Unique GID List
# X_train_GID_sample <- as.data.frame(X_train_GID[train_random_sample,], row.names = row.names(X_train_GID)[train_random_sample])
# colnames(X_train_GID_sample) <- c('GID')
# X_test_GID_sample <- as.data.frame(X_test_GID[test_random_sample,], row.names = row.names(X_test_GID)[test_random_sample])
# colnames(X_test_GID_sample) <- c('GID')

In [ ]:
# # Combine Train and Test but keep index of separation
# y_sample <- rbind(y_train_sample,y_test_sample)
# W_CIMMYT_sample <- rbind(W_CIMMYT_train_sample, W_CIMMYT_test_sample)
# W_gee_sample <- rbind(W_gee_train_sample, W_gee_test_sample)
# X_GID_sample <- rbind(X_train_GID_sample, X_test_GID_sample)
# print(length(y_sample[train_random_sample,]))
# print(length(y_sample[test_random_sample,]))
# print(length(X_GID_sample[train_random_sample,]))
# print(length(X_GID_sample[test_random_sample,]))

In [12]:
# Combine Train and Test but keep index of separation
y <- rbind(as.data.frame(y_train),as.data.frame(y_test))
W_CIMMYT <- rbind(as.data.frame(W_CIMMYT_train), as.data.frame(W_CIMMYT_test))
# W_gee_sample <- rbind(W_gee_train_sample, W_gee_test_sample)
X_GID <- rbind(as.data.frame(X_train_GID), as.data.frame(X_test_GID))

In [ ]:
head(y)
print(dim(y))

In [ ]:
head(W_CIMMYT)
print(dim(W_CIMMYT))

In [ ]:
head(X_GID)
print(dim(X_GID))

In [13]:
#set.seed(123)
#training.samples <- y_sample$GRAIN_YIELD %>%
#  createDataPartition(p = 0.9, list = FALSE)

test_id_pos = y[rownames(y_test),]
head(test_id_pos)

[1] -0.5217354 -0.5868307  0.5826089  0.0222429 -0.1461855  0.3941511

In [14]:
train_rows = nrow(y_train)

In [15]:
y_train_na = y$GRAIN_YIELD
y_train_na[-seq_len(train_rows)] = NA

In [20]:
head(y_train_na)

[1] -0.6984829 -0.5272796 -0.7130147 -0.7016617 -0.5654257 -0.3152056

In [21]:
length(y_train_na)

[1] 144099

In [22]:
head(y_train_na[seq_len(train_rows)])

[1] -0.6984829 -0.5272796 -0.7130147 -0.7016617 -0.5654257 -0.3152056

In [23]:
head(y_train_na[-seq_len(train_rows)])

[1] NA NA NA NA NA NA

## Model 1: Main Variety Effects

In [17]:
# Fitting Model 1 (Main Environment-Line Effects)
# (EL)

# incidence matrix for main eff. of environments.
# ZE = model.matrix(~factor(Y$ENV)-1)     

# incidence matrix for main eff. of lines.
X_GID$GID = factor(x=X_GID$GID,levels=rownames(A),ordered=TRUE)
ZVAR = model.matrix(~X_GID$GID-1)
print(dim(ZVAR))
print(length(y_train_na))

[1] 144099   3489
[1] 144099


In [ ]:
# Model Fitting

# ETA = list(ENV=list(X=ZE,model='BRR'),
#            VAR=list(X=ZVAR,model='BRR'))

ETA = list(VAR=list(X=ZVAR,model='BRR'))

fm1 = BGLR(y=y_train_na,ETA=ETA,saveAt='M1_',nIter=1000,burnIn=500)

In [ ]:
head(fm1$yHat[-seq_len(train_rows)])

In [ ]:
head(y$GRAIN_YIELD[-seq_len(train_rows)])

In [ ]:
predictions_1 = fm1$yHat[-seq_len(train_rows)]
test_values = y$GRAIN_YIELD[-seq_len(train_rows)]
R2 = R2(predictions_1, test_values)
RMSE = RMSE(predictions_1, test_values)
MAE = MAE(predictions_1, test_values)
model1_scores = data.frame(R2, RMSE, MAE)
print(model1_scores)
plot(test_values,predictions_1)

## Model 2: Main Pedigree Effects 

In [18]:
# Alternative Model 2 with Eigendecomposition
A = as.matrix(A)/mean(diag(as.matrix(A)))
L_star = eigen(A, symmetric =TRUE)
Gamma = L_star$vectors
Lambda = L_star$values 
Z_star = ZVAR%*%Gamma%*%(Lambda^0.5)

In [ ]:
# Fitting Model 2 with Alternative Z_star (Main Environment-Pedigree Effects)
# (EA)

# ETA = list(ENV=list(X=ZE,model='BRR'),
#            PED=list(X=Z_star,model='BRR'))

ETA = list(PED=list(X=Z_star,model='BRR'))

In [ ]:
fm2 = BGLR(y=y_train_na,ETA=ETA,saveAt='M2_',nIter=2000,burnIn=500)

In [ ]:
predictions_2 = fm2$yHat[-seq_len(train_rows)]
test_values = y$GRAIN_YIELD[-seq_len(train_rows)]
R2 = R2(predictions_2, test_values)
RMSE = RMSE(predictions_2, test_values)
MAE = MAE(predictions_2, test_values)
model2_scores = data.frame(R2, RMSE, MAE)
print(model2_scores)
plot(test_values,predictions_2)

## Model 3: Just Environment Main Effects

In [19]:
W_CIMMYT_div = W_CIMMYT/sqrt(ncol(W_CIMMYT))

In [ ]:
# Model 3 with just EC - CIMMYT
ETA = list(EC=list(X=W_CIMMYT_div,model='BRR'))

In [ ]:
fm3_ec_CIMMYT = BGLR(y=y_train_na,ETA=ETA,saveAt='M3_',nIter=2000,burnIn=500)

In [ ]:
predictions_3_CIMMYT = fm3_ec_CIMMYT$yHat[-seq_len(train_rows)]
test_values = y$GRAIN_YIELD[-seq_len(train_rows)]
R2 = R2(predictions_3_CIMMYT, test_values)
corr = cor(predictions_3_CIMMYT, test_values)
RMSE = RMSE(predictions_3_CIMMYT, test_values)
MAE = MAE(predictions_3_CIMMYT, test_values)
model3_CIMMYT_scores = data.frame(R2, RMSE, MAE)
print(model3_CIMMYT_scores)
plot(test_values,predictions_3_CIMMYT)

In [ ]:
W_gee_sample_div = W_gee_sample/sqrt(ncol(W_gee_sample))

In [ ]:
# Model 3 with just EC - GEE
ETA = list(EC=list(X=W_gee_sample_div,model='BRR'))

In [ ]:
fm3_ec_gee = BGLR(y=y_train_na,ETA=ETA,saveAt='M3_',nIter=500,burnIn=200)

In [ ]:
predictions_3_gee = fm3_ec_gee$yHat[-seq_len(train_rows)]
test_values = y_sample$GRAIN_YIELD[-seq_len(train_rows)]
R2 = R2(predictions_3_gee, test_values)
corr = cor(predictions_3_gee, test_values)
RMSE = RMSE(predictions_3_gee, test_values)
MAE = MAE(predictions_3_gee, test_values)
model3_CIMMYT_scores = data.frame(R2, RMSE, MAE)
print(model3_CIMMYT_scores)
plot(test_values,predictions_3_gee)

## Model 4: Main Pedigree and Environment Effects

In [ ]:
# Fitting Model 3_CIMMYT (Main Environment-Pedigree-Environmental Covariate Effects)
# (EAW)

In [ ]:
ETA = list(PED=list(X=Z_star,model='BRR'),
           EC=list(X=W_CIMMYT_div,model='BRR'))

In [ ]:
fm4_CIMMYT = BGLR(y=y_train_na,ETA=ETA,saveAt='M4_CIM_',nIter=2000,burnIn=500)

In [ ]:
predictions_4_CIMMYT = fm4_CIMMYT$yHat[-seq_len(train_rows)]
test_values = y$GRAIN_YIELD[-seq_len(train_rows)]
R2 = R2(predictions_4_CIMMYT, test_values)
RMSE = RMSE(predictions_4_CIMMYT, test_values)
MAE = MAE(predictions_4_CIMMYT, test_values)
model4_CIMMYT_scores = data.frame(R2, RMSE, MAE)
print(model4_CIMMYT_scores)
plot(test_values,predictions_4_CIMMYT)

In [ ]:
# Fitting Model 3_GEE (Main Environment-Pedigree-Environmental Covariate Effects)
# (EAW)

W_gee_sample_div = W_gee_sample/sqrt(ncol(W_gee_sample))

In [1]:
ETA = list(PED=list(X=Z_star,model='BRR'),
           EC=list(X=W_gee_sample_div,model='BRR'))

ERROR: Error in eval(expr, envir, enclos): object 'Z_star' not found


In [ ]:
fm4_gee = BGLR(y=y_train_na,ETA=ETA,saveAt='M3_',nIter=500,burnIn=200)


In [ ]:
predictions_4_gee = fm4_gee$yHat[-seq_len(train_rows)]
test_values = y_sample$GRAIN_YIELD[-seq_len(train_rows)]
R2 = R2(predictions_4_gee, test_values)
RMSE = RMSE(predictions_4_gee, test_values)
MAE = MAE(predictions_4_gee, test_values)
model4_gee_scores = data.frame(R2, RMSE, MAE)
print(model4_gee_scores)
plot(test_values,predictions_4_gee)

## Model 5: Main Pedigree, Main Environment, and Interaction Effects

In [22]:
gc()

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2218363,118.5,7152185,382.0,5393861,288.1
Vcells,42097751590,321180.4,60623354108,462519.5,42097813263,321180.9


In [29]:
get_ram()

1.02 TB

In [20]:
# Fitting Model 5 (Main Environment-Pedigree-Environmental Covariate Effects 
# with Pedigree-Environmental Covariate Interaction)
# (EAW-AxW) CIMMYT

# Results in vector of size 157.4 Gb, need sufficient RAM to hold

ZAZ = tcrossprod(Z_star)

In [21]:
W_matrix = as.matrix(W_CIMMYT_div)

WW = tcrossprod(W_matrix)

In [23]:
K = ZAZ*WW
diag(K) = diag(K)+1/200 
K = K/mean(diag(K))

In [ ]:
# L2 = t(chol(K))
L_5_star = eigen(K, symmetric =TRUE)
Gamma_5 = L_5_star$vectors
Lambda_5 = L_5_star$values 

In [ ]:
L2 = Gamma_5%*%(Lambda_5^0.5)

In [ ]:
ETA = list(PED=list(X=Z_star,model='BRR'),
           EC=list(X=W_matrix,model='BRR'),
           AxW=list(X=L2,model='BRR'))

In [ ]:
fm5_CIMMYT = BGLR(y=y_train_na,ETA=ETA, saveAt='M4_',nIter=200,burnIn=500)

In [ ]:
predictions_5_CIMMYT = fm5_CIMMYT$yHat[-seq_len(train_rows)]
test_values = y_sample$GRAIN_YIELD[-seq_len(train_rows)]
corr = cor(predictions_5_CIMMYT, test_values)
R2 = R2(predictions_5_CIMMYT, test_values)
RMSE = RMSE(predictions_5_CIMMYT, test_values)
MAE = MAE(predictions_5_CIMMYT, test_values)
model5_CIMMYT_scores = data.frame(R2, corr, RMSE, MAE)
print(model5_CIMMYT_scores)
plot(test_values,predictions_5_CIMMYT)

In [ ]:
remove(L2, W_matrix, K, WW)
# Garbage Collector to Clear up RAM
gc()

In [ ]:
# Fitting Model 5 (Main Environment-Pedigree-Environmental Covariate Effects 
# with Pedigree-Environmental Covariate Interaction)
# (EAW-AxW) GEE

# ZAZ = tcrossprod(Z_star) # Already run above

W_matrix = as.matrix(W_gee_sample_div)

WW = tcrossprod(W_matrix)

K = ZAZ*WW
diag(K) = diag(K)+1/200 
K = K/mean(diag(K))
L2 = t(chol(K))

In [ ]:
ETA = list(PED=list(X=Z_star,model='BRR'),
           EC=list(X=W_matrix,model='BRR'),
           AxW=list(X=L2,model='BRR'))

In [ ]:
fm5_gee = BGLR(y=y_train_na,ETA=ETA, saveAt='M4_',nIter=500,burnIn=200)

In [ ]:
predictions_5_GEE = fm5_gee$yHat[-seq_len(train_rows)]
test_values = y_sample$GRAIN_YIELD[-seq_len(train_rows)]
R2 = R2(predictions_5_GEE, test_values)
corr = cor(predictions_5_GEE, test_values)
RMSE = RMSE(predictions_5_GEE, test_values)
MAE = MAE(predictions_5_GEE, test_values)
model5_GEE_scores = data.frame(R2, corr, RMSE, MAE)
print(model5_GEE_scores)
plot(test_values,predictions_5_GEE)